In [2]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
from scipy.constants import lbf, inch, mph, mach
from dataclasses import dataclass
import matplotlib.pyplot as plt

We define our terms as follows:

vehicle state: $x$

control vector: $u$

dynamics: $\dot{x} = f(x,u,t)$

observation: $y = h(x,t)$

control scheme: $c(y)$

combined equation: $\dot{x} = f(x, c(h(x,t)),t)$

since the true dynamics are unknown, we say $f\in F$ where $F$ is a space of possible dynamics.  We define our best-guess dynamics as $f_0\in F$ and define a metric on the space of dynamics $d(f_1,f_2):f_1,f_2\in F$.

We define $L(f,c)$ to be the maximum error from target acceleration of the rocket during flight with dynamics $f$ and control system $c$.

We formulate our problem as "Choose c as to maximize the minimum distance $d(f_0, f)$ such that $L(f,c)\geq0.1$"



In [5]:
def parse_apc_propeller_data(filename):
    
    points = []
    values = []
    
    f = open(filename)
        
    tables = f.read().split('PROP RPM = ')
        
    for table in tables[1:]:
        rows = table.split('\n')        
        #Find RPM
        rpm = int(rows[0].split()[0])
        
        header = rows[2].split()
        velocity_index = header.index('V')
        torque_index = header.index('Torque')
        thrust_index = header.index('Thrust')
                
        for row in rows[4:-4]:
            entrees = row.split()
            velocity = float(entrees[velocity_index]) * mph
            torque_x = float(entrees[torque_index]) * lbf * inch
            thrust_x = float(entrees[thrust_index]) * lbf
            points.append([rpm, velocity])
            values.append(np.array([thrust_x, 0, 0, torque_x, 0, 0]))
            
    return LinearNDInterpolator(points, values, 0)

def parse_drag_data(filename):
    
    points = []
    values = []
    
    f = open(filename)
    
    rows = f.read().split('\n')
    for row in rows[1:]:
        entrees = row.split(', ')
        mach = float(entrees[0])
        drag_coef = float(entrees[1])
        
        points.append(mach)
        values.append(drag_coef)
        
    return LinearNDInterpolator(points, values)
    
rpm_vel_to_wrench = parse_apc_propeller_data('./data/PER3_5x75E.dat')
mach_to_cd = parse_drag_data('./data/drag_estimate.txt')

print(thrust_curve(1000, 0))



def control_system(target, observation, memory):
     
    kp = 0.1
    ki = 0.0
    kd = 0.0
    
    p = target - observation
    i = memory['i'] + p
    d = p - memory['p']
    
    signal = kp * p + ki * i + kd * d
    
    memory['p'] = p
    memory['i'] = i
    
    return signal


ValueError: could not convert string to float: ''